Update data

- CHECK for additional countries to include
- Pull data from online sources (OWID?)
- Process and format as needed
- Export to .vdf

Re-run basic calibration with updated .vdf files

- Back up previous day's .out file
- With fixed .voc and .vpd etc., reading output from previous day's calibration?

Export calibration results

- Read .out file, run with each .lst file
- Export VDF with each .lst file to CSV

In [1]:
import os
import subprocess
import re
import json
import time
import pandas as pd
import numpy as np

from keyboard import press
from shutil import copy
from datetime import date

In [2]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

class Script(object):
    """Master object for holding and modifying .cmd script settings, 
    creating .cmd files, and running them through Vensim/Vengine"""
    def __init__(self, controlfile):
        print("Initialising", self)
        for k, v in controlfile['simsettings'].items():
            self.__setattr__(k, v if isinstance(v, str) else v.copy())
        self.runcmd = "MENU>RUN_OPTIMIZE|o\n"
        self.savecmd = f"MENU>VDF2TAB|!|!|{self.savelist}|\n"
        self.basename = controlfile['baserunname']
        self.cmdtext = []
        
    def copy_model_files(self, dirname):
        """Create subdirectory and copy relevant model files to it,
        then change working directory to subdirectory"""
        os.makedirs(dirname, exist_ok=True)
        os.chdir(f"./{dirname}")

        # Copy needed files from the working directory into the sub-directory
        for s in ['model', 'payoff', 'optparm', 'sensitivity', 'savelist', 'senssavelist']:
            if getattr(self, s):
                copy(f"../{getattr(self, s)}", "./")
        for slist in ['data', 'changes']:
            for file in getattr(self, slist):
                copy(f"../{file}", "./")
            
    def add_suffixes(self, settingsfxs):
        for s, sfx in settingsfxs.items():
            if hasattr(self, s):
                self.__setattr__(s, getattr(self, s)[:-4] + sfx + getattr(self, s)[-4:])
   
    def update_changes(self, chglist):
        # Combines and flattens list of paired change names & suffixes
        flatlist = [i for s in 
                    [[f"{self.basename}_{n}_{sfx}.out" for n in name] if isinstance(name, list) 
                     else [f"{self.basename}_{name}_{sfx}.out"] for name, sfx in chglist] for i in s]
        self.changes.extend(flatlist)
          
    def write_script(self, scriptname):
        self.cmdtext.extend(["SPECIAL>NOINTERACTION\n", 
                             f"SPECIAL>LOADMODEL|{self.model}\n"])
        
        for s in ['payoff', 'sensitivity', 'optparm', 'savelist', 'senssavelist']:
            if hasattr(self, s):
                self.cmdtext.append(f"SIMULATE>{s}|{getattr(self, s)}\n")
        
        if hasattr(self, 'data'):
            datatext = ','.join(self.data)
            self.cmdtext.append(f"SIMULATE>DATA|\"{','.join(self.data)}\"\n")

        if hasattr(self, 'changes'):
            self.cmdtext.append(f"SIMULATE>READCIN|{self.changes[0]}\n")
            for file in self.changes[1:]:
                self.cmdtext.append(f"SIMULATE>ADDCIN|{file}\n")
        
        self.cmdtext.extend(["\n", f"SIMULATE>RUNNAME|{scriptname}\n", 
                             self.runcmd, self.savecmd, 
                             "SPECIAL>CLEARRUNS\n", "MENU>EXIT\n"])
        
        with open(f"{scriptname}.cmd", 'w') as scriptfile:
            scriptfile.writelines(self.cmdtext)
    
    def run_script(self, scriptname, controlfile, subdir, logfile):
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit'], '.log', check_opt, logfile)

    
class SubdirScript(Script):
    """Script subclass for optimization runs in subdirectory"""
    def prep_subdir(self, scriptname, controlfile, subdir):
        self.copy_model_files(subdir)
        copy(f"../{scriptname}.cmd", "./")
        for file in self.changes:
            if file[-4:] == '.out':
                clean_outfile(file, [f"[{subdir}"])

    def run_script(self, scriptname, controlfile, subdir, logfile):
        """Copy necessary files to subdirectory and run there, copying 
        .out file back to parent directory when complete"""
        self.prep_subdir(scriptname, controlfile, subdir)
        payoff = run_vengine_script(scriptname, controlfile['vensimpath'], 
                                    controlfile['timelimit'], '.log', check_opt, logfile)
        copy(f"./{scriptname}.out", "..") # Copy the .out file to parent directory
        os.chdir("..")
        return payoff    

    
class LongScript(Script):
    """Script subclass for long calibration runs e.g. all-params"""
    def run_script(self, scriptname, controlfile, subdir, logfile):
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit']*20, '.log', check_opt, logfile)

    
class RunScript(Script):
    """Script subclass for simple single runs (not optimzations)"""
    def __init__(self, controlfile):
        super().__init__(controlfile)
        self.runcmd = "MENU>RUN|o\n"
    
    def run_script(self, scriptname, controlfile, subdir, logfile):
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  10, '.vdf', check_run, logfile)


def compile_script(controlfile, scriptclass, name, namesfx, settingsfxs, 
                   logfile, chglist=[], subdir=None):
    """Master function for assembling & running .cmd script
    
    Parameters
    ----------
    controlfile : JSON object
        Master control file specifying sim settings, runname, etc.
    scriptclass : Script object
        Type of script object to instantiate, depending on run type
    name : str
    namesfx : str
        Along with `name`, specifies name added to baserunname for run
    settingsfxs : dict of str
        Dict of suffixes to append to filenames in simsettings; use to 
        distinguish versions of e.g. .mdl, .voc, .vpd etc. files
    logfile : str of filename/path
    chglist : list of tuples of (str or list, str)
        Specifies changes files to be used in script; specify as tuples 
        corresponding to `name`, `namesfx` of previous run .out to use; 
        tuples can also take a list of `names` as first element, taking 
        each with the same second element; `chglist` can also take one 
        non-tuple str as its last element, which will be added directly 
        (e.g. for policy scenario .cin files)
    subdir : str, optional
        Name of subdirectory to create/use for run, if applicable
    
    Returns
    -------
    float
        Payoff value of the script run, if applicable, else 0
    """
    mainscript = scriptclass(controlfile)
    mainscript.add_suffixes(settingsfxs)
    mainscript.update_changes(chglist)
    scriptname = f"{mainscript.basename}_{name}_{namesfx}"    
    mainscript.write_script(scriptname)
    return mainscript.run_script(scriptname, controlfile, subdir, logfile)


def write_log(string, logfile):
    """Writes printed script output to a logfile"""
    with open(logfile,'a') as f:
        f.write(string + "\n")
    print(string)
    

def check_opt(scriptname, logfile):
    """Check function for use with run_vengine_script for optimizations 
    WARNING: will trigger if ANY .voc parameters have lower bounds at 0 - 
    to avoid, ensure lower bounds are some small value instead e.g. 1e-06"""
    if check_zeroes(scriptname):
        write_log(f"Help! {scriptname} is being repressed!", logfile)
    return not check_zeroes(scriptname)

def check_MC(scriptname, logfile, threshold=0.1):
    """Check function for use with run_vengine_script for MCMC"""
    if abs(compare_payoff(scriptname, logfile)) >= threshold:
        write_log(f"{scriptname} is a self-perpetuating autocracy! re-running MC...", logfile)
        return False
    return True

def check_run(scriptname, logfile):
    """Check function for use with run_vengine_script for normal & sens runs"""
    if not os.path.exists(f"./{scriptname}.vdf"):
        write_log(f"Help! {scriptname} is being repressed!", logfile)
    return os.path.exists(f"./{scriptname}.vdf")

def run_vengine_script(scriptname, vensimpath, timelimit, checkfile, check_func, logfile):
    """Call Vensim with command script using subprocess; monitor output 
    file for changes to see if Vensim has stalled out, and restart if 
    it does, or otherwise bugs out; return payoff if applicable"""

    write_log(f"Initialising {scriptname}!", logfile)
    
    while True:
        proc = subprocess.Popen(f"{vensimpath} \"./{scriptname}.cmd\"")
        time.sleep(2)
        press('enter') # Necessary to bypass the popup message in Vengine
        while True:
            try:
                proc.wait(timeout=timelimit)
                break
            except subprocess.TimeoutExpired:
                try:
                    write_log(f"Checking for {scriptname}{checkfile}...", logfile)
                    timelag = time.time() - os.path.getmtime(f"./{scriptname}{checkfile}")
                    if timelag < (timelimit):
                        write_log(f"At {time.ctime()}, {round(timelag,3)}s since last output, "
                                  "continuing...", logfile)
                        continue
                    else:
                        proc.kill()
                        write_log(f"At {time.ctime()}, {round(timelag,3)}s since last output. "
                                  "Calibration timed out!", logfile)
                        break
                except FileNotFoundError:
                    proc.kill()
                    write_log("Calibration timed out!", logfile)
                    break
        if proc.returncode != 1: # Note that Vengine returns 1 on MENU>EXIT, not 0!
            write_log(f"Return code is {proc.returncode}", logfile)
            write_log("Vensim! Trying again...", logfile)
            continue
        try:
            if check_func(scriptname, logfile):
                break
        except FileNotFoundError:
            write_log("Outfile not found! That's it, I'm dead.", logfile)
            pass
    
    time.sleep(2)

    if os.path.exists(f"./{scriptname}.out"):
        payoffvalue = read_payoff(f"{scriptname}.out")
        write_log(f"Payoff for {scriptname} is {payoffvalue}, calibration complete!", logfile)
        return payoffvalue
    return 0 # Set default payoff value for simtypes that don't generate one


def check_zeroes(scriptname):
    """Check if an .out file has any parameters set to zero (indicates Vengine error),
    return True if any parameters zeroed OR if # runs = # restarts, and False otherwise"""
    filename = f"{scriptname}.out"
    with open(filename,'r') as f0:
        filedata = f0.readlines()
    
    checklist = []
    for line in filedata:
        if line[0] != ':':
            if ' = 0 ' in line:
                checklist.append(True)
            else:
                checklist.append(False)
        elif ':RESTART_MAX' in line:
            restarts = re.findall(r'\d+', line)[0]
    
    # Ensure number of simulations != number of restarts
    if f"After {restarts} simulations" in filedata[0]:
        checklist.append(True)
    
    return any(checklist)


def read_payoff(outfile, line=1):
    """Identifies payoff value from .OUT or .REP file - 
    use line 1 (default) for .OUT, or use line 0 for .REP"""
    with open(outfile) as f:
        payoffline = f.readlines()[line]
    payoffvalue = [float(s) for s in re.findall(r'-?\d+\.?\d+[eE+-]*\d+', payoffline)][0]
    return payoffvalue


def compare_payoff(scriptname, logfile):
    """Returns the difference in payoffs between .OUT and .REP file, 
    which should be zero in most cases except when MCMC bugs out"""
    difference = read_payoff(f"{scriptname}.out") - read_payoff(f"{scriptname}.rep", 0)
    write_log(f".OUT and .REP payoff difference is {difference}", logfile)
    return difference


def modify_mdl(country, modelname, newmodelname):
    """Opens .mdl as text, identifies Rgn subscript, and replaces 
    with appropriate country name"""
    with open(modelname,'r') as f:
        filedata = f.read()
        
    rgnregex = re.compile(r"Rgn(\s)*?:(\n)?[\s\S]*?(\n\t~)")
    newdata = rgnregex.sub(f"Rgn:\n\t{country}\n\t~", filedata)

    with open(newmodelname,'w') as f:
        f.write(newdata)
    
                       
def split_voc(vocname):
    """Splits .VOC file into multiple versions, for main, country, initial, 
    full model, general MCMC, and country MCMC calibration"""
    with open(vocname,'r') as f0:
        filedata = f0.readlines()
    
    vocmain = [line for line in filedata if line[0] == ':' or '[Rgn]' not in line]
    voccty = [line for line in filedata if line[0] == ':' or '[Rgn]' in line]
    vocfull = filedata.copy()
    vocinit = voccty.copy()
    
    # Set restarts to 1 for vocs besides initial
    for voc in (vocmain, voccty, vocfull):
        for l, line in enumerate(voc):
            if ':RESTART_MAX' in line:
                voc[l] = ':RESTART_MAX=1\n'
            
    # Write various voc versions to separate .voc files
    for fname, suffix in zip([vocmain, voccty, vocinit, vocfull], 
                             ['m', 'c', 'i', 'f']):
        with open(f"{vocname[:-4]}_{suffix}.voc", 'w') as f:
            f.writelines(fname)

    
def clean_outfile(outfilename, linekey):
    """Clean an outfile to include only lines containing a string in [linekey]
    Note that [linekey] should be a list of strings to keep"""
    with open(outfilename,'r') as f:
        filedata = f.readlines()

    newdata = [line for line in filedata if any(k in line for k in linekey)]
    
    with open(outfilename, 'w') as f:
        f.writelines(newdata)
            
            
def create_mdls(controlfile, mainlist, sublist, logfile):
    """Creates copies of the base .mdl file for each country in list (and one main copy)
    and splits .VOC files"""
    model = controlfile['simsettings']['model']
    for c in sublist:
        newmodel = model[:-4] + f'_{c}.mdl'
        modify_mdl(c, model, newmodel)

    mainmodel = model[:-4] + '_main.mdl'
    c_list = [f'{c}\\\n\t\t' if i % 8 == 7 else c for i,c in enumerate(mainlist)]
    countrylist_str = str(c_list)[1:-1].replace("'","")
    modify_mdl(countrylist_str, model, mainmodel)
    split_voc(controlfile['simsettings']['optparm'])
    write_log("Files are ready! moving to calibration", logfile)


def get_first_idx(s):
        return (s > 0).idxmax(skipna=True)


def import_datasets(datalist, vdfname):
    """Creates Vensim script to convert CSVs to VDFs"""
    print("Importing data to VDF...")
    scenario_text = []
    scenario_text.append("SPECIAL>NOINTERACTION\n")
    
    for dataname in datalist:
        scenario_text.append(f"MENU>CSV2VDF|{dataname}.csv|{vdfname}{dataname}.vdf|{dataname}.frm|\n")
    
    scenario_text.append("MENU>EXIT\n")
    
    scriptfile = open("ImportData.cmd", 'w')
    scriptfile.writelines(scenario_text)
    scriptfile.close()
    

def export_datasets(savelists, vdfname):
    """Creates Vensim script to export VDFs to CSVs with each savelist specified"""
    print("Exporting data to CSV...")
    scenario_text = []
    scenario_text.append("SPECIAL>NOINTERACTION\n")
    
    for savelist in savelists[0:2]:
        scenario_text.append(f"MENU>VDF2CSV|{vdfname}.vdf|{savelist}.csv|{savelist}.lst||1|\n")
    for savelist in savelists[2:]:
        scenario_text.append(f"MENU>VDF2CSV|{vdfname}.vdf|{savelist}.csv|{savelist}.lst|*!|1|\n")
    
    scenario_text.append("MENU>EXIT\n")
    
    scriptfile = open("ExportData.cmd", 'w')
    scriptfile.writelines(scenario_text)
    scriptfile.close()


In [3]:
controlfilename = input("Enter control file name (with extension):")
cf = json.load(open(controlfilename, 'r'))

# Unpack controlfile into variables
for k,v in cf.items():
    exec(k + '=v')
    
for k, v in datasettings.items():
    exec(k + '=v')

logfile = f"{os.getcwd()}/{baserunname}.log"
write_log(f"-----\nStarting new log at {time.ctime()}\nReady to work!", logfile)


Enter control file name (with extension):V68Updates.txt
-----
Starting new log at Tue Aug 31 14:06:40 2021
Ready to work!


In [ ]:
##### THIS CELL IS FOR UPDATING DATA ONLY #####

data = pd.read_csv(data_url) # Read data from URL for raw data CSV

# Subset CSV to relevant data fields
data = data.filter(['location', 'date', 'total_cases', 'new_cases_smoothed', 'total_deaths', 
                    'new_deaths_smoothed', 'total_tests', 'new_tests_smoothed', 
                    'people_vaccinated'], axis=1)

# Rename fields as needed
data.columns = ['location','date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 
                'total_tests', 'new_tests', 'total_vaccinations']

table = pd.pivot_table(data, values=['total_cases', 'new_cases', 'total_deaths', 'new_deaths', 
                                     'total_tests', 'new_tests', 'total_vaccinations'], 
                       index='date', columns='location')
table = table.T

# Calculate daily new people vaccinated from cumulative people vaccinated
new_vaccinations = pd.concat({'new_vaccinations': table.loc['total_vaccinations'].diff(axis=1)})
table = table.append(new_vaccinations)

# Label indices and change columns to numeric form
table.index.names = ['field', 'location']
table.columns = (pd.to_datetime(table.columns) - pd.to_datetime(startdate)).days
table = table.rename(index=renames)
table[table <= 0] = np.NaN # Drop negative values

# Reorder multiindex levels before by-country subsetting
table = table.reorder_levels(['location', 'field']).sort_index()
table = table.loc[countrylist]
table = table.reorder_levels(['field', 'location']).sort_index()

# pd.set_option('display.max_rows', None)
# display(table)

no_tottest_idx = [c for c in countrylist if c not in table.loc['total_tests'].index]

total_tests = table.loc['total_tests']

for c in no_tottest_idx:
    infA = get_first_idx(table.loc['new_cases'].loc[c])
    testA = get_first_idx(table.loc['new_tests'].loc[c])
    
    if infA < testA:
        table.loc['new_tests'].loc[c, infA] = 1
    
    table.loc['new_tests'].loc[c] = table.loc['new_tests'].loc[c].interpolate(limit_area='inside')
    tottests = table.loc['new_tests'].loc[c].cumsum()
    total_tests = total_tests.append(tottests)

total_tests.sort_index(inplace=True)

for c in total_tests.index:    
    total_tests.loc[c] = total_tests.loc[c].mask(total_tests.loc[c].cummax().duplicated())
    
    infA = get_first_idx(table.loc['new_cases'].loc[c])
    testA = get_first_idx(total_tests.loc[c])
    
    if infA < testA:
        total_tests.loc[c, infA] = 1
    testA = min(infA, testA)
    
    total_tests.loc[c, :testA] = 0
    
total_tests.interpolate('pchip', axis=1, limit_area='inside', inplace=True)

new_tests = total_tests.diff(axis=1)

testdf = pd.concat([total_tests, new_tests], keys=['total_tests', 'new_tests'])
testdf = testdf.iloc[:, 1:] # Drop first column
flowdf = table.loc[['new_cases', 'new_deaths', 'new_vaccinations']]
formdf = table.loc[['total_cases', 'total_deaths', 'total_vaccinations']]

for df, name in zip([flowdf, formdf, testdf], ['FlowData', 'FormattedData', 'TestData']):
    df.rename(datavarnames, inplace=True)
    df.index.names = ['Time', 'Rgn']
    df.to_csv(f'{name}.csv')

display(testdf)
display(flowdf)
display(formdf)

In [ ]:
import_datasets(datalist, vdfname)
subprocess.run(f"{vensim7path} \"./ImportData.cmd\"", check=True)

In [ ]:
# Update time bounds based on current data availability
finaltime = table.columns[-1]
with open(simsettings['changes'][0], 'w') as f: # Takes first file in changes setting!
    f.write(f"FINAL TIME = {finaltime}\nMax Time Data Used = {finaltime}")

# Create submodels for specific countries to pre-calibrate
create_mdls(cf, countrylist, precal_list, logfile)

# First do any country-specific pre-calibration
for c in precal_list:
    compile_script(cf, SubdirScript, c, 0, {'model' : f'_{c}', 'optparm' : '_i'}, logfile, subdir=c)

# Then do full calibration
compile_script(cf, LongScript, 'main', 'full', {'optparm': '_f'}, logfile, chglist=[(precal_list, 0)])

In [4]:
export_datasets(savelists, 'Updated_main_full')

# while True:
proc = subprocess.Popen(f"{vensim7path} \"./ExportData.cmd\"")
#     proc.wait()
#     if proc.returncode == 0:
#         break
#     else: continue

# Calculate preceding date and backup .out file and data/output CSVs
write_log("Backing up files...", logfile)
date_str = date.today().strftime("%Y%m%d")
os.makedirs(f'./{date_str}', exist_ok=True)
copy('Updated_main_full.out', 
     f'./{date_str}/Updated_{date_str}.out')
for csvname in (datalist + savelists):
    copy(f'{csvname}.csv', f'./{date_str}/{csvname}_{date_str}.csv')

# Copy output CSVs and VDF to Stella working folder
for csvname in savelists:
    copy(f'{csvname}.csv', '../Data')
copy('Updated_main_full.out', '../Data')

write_log(f"Log completed at {time.ctime()}. Job done!", logfile)

Exporting data to CSV...
Backing up files...
Log completed at Tue Aug 31 14:06:44 2021. Job done!


In [ ]:
"""

Backup previous .out file

Compile script and run

for savelist in savelists - change savelist then run as runscript

"""